In [1]:
import sys
sys.path.insert(2, '/home/gaurang/glycemic_control/code/src/')

In [2]:
import argparse
import gc
import logging
import time
from pathlib import Path
from typing import Any, Callable, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader

from src.constants import CONTINUOUS_COVARIATES_PROCESSED, STATIC_COLS, TARGET_COL
from src.dataset import TransformerDataset, df_to_patient_tensors, get_normalizing_scaler
from src.models.transformer import TimeSeriesTransformer
from src.utils import (
    EarlyStopping,
    generate_square_subsequent_mask,
    get_patient_indices,
    get_timestamp,
)

GLOBAL_SEED = 123
device = torch.device("cpu")

In [3]:
fpath_train = "/home/gaurang/glycemic_control/data/glycaemia_project_csvs/processed_data/train_test_splits/v1_patient_split_mini/train_mini.csv"
fpath_valid = "/home/gaurang/glycemic_control/data/glycaemia_project_csvs/processed_data/train_test_splits/v1_patient_split_mini/valid_mini.csv"
fpath_test = "/home/gaurang/glycemic_control/data/glycaemia_project_csvs/processed_data/train_test_splits/v1_patient_split_mini/test_mini.csv"

In [4]:
df_train = pd.read_csv(fpath_train)
df_valid = pd.read_csv(fpath_valid)
df_test = pd.read_csv(fpath_test)

In [5]:
np.random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)

# Hyperparams
batch_size = 4
lr = 1e-4
num_epochs = 20

# Params
enc_seq_len = 3  # length of input given to encoder
output_sequence_length = 1  # how many future glucose values to predict
step_size = 1  # Step size, i.e. how many time steps does the moving window move at each step
batch_first = True

# Define input variables
exogenous_vars = (
    CONTINUOUS_COVARIATES_PROCESSED + STATIC_COLS
)  # Each element must correspond to a column name
input_variables = TARGET_COL + exogenous_vars

print(
    f"Time series params: \nInput sequence lenght: {enc_seq_len} \nOutput sequence lenght:"
    f" {output_sequence_length} \nStep size: {step_size}"
)

print(f"Model hyperparameters: \nBatch size: {batch_size} \nLearning rate: {lr}")

# df to patient tensor
scaler = get_normalizing_scaler(df_train[input_variables])
X_train, y_train = df_to_patient_tensors(
    df_train, feature_cols=input_variables, target_col=TARGET_COL, scaler=scaler
)
X_valid, y_valid = df_to_patient_tensors(
    df_valid, feature_cols=input_variables, target_col=TARGET_COL, scaler=scaler
)
X_test, y_test = df_to_patient_tensors(
    df_test, feature_cols=input_variables, target_col=TARGET_COL, scaler=scaler
)

# get subsequence indices
indices_train, num_samples_train = get_patient_indices(
    y_train, input_seq_len=enc_seq_len, forecast_len=output_sequence_length, step_size=step_size
)
indices_valid, num_samples_valid = get_patient_indices(
    y_valid, input_seq_len=enc_seq_len, forecast_len=output_sequence_length, step_size=step_size
)
indices_test, num_samples_test = get_patient_indices(
    y_valid, input_seq_len=enc_seq_len, forecast_len=output_sequence_length, step_size=step_size
)
print(
    f"Number of training samples: {num_samples_train}"
    f" \nNumber of valid samples: {num_samples_valid}"
    f" \nNumber of test samples: {num_samples_test}"
)

# create datasets

dataset_train = TransformerDataset(
    data=X_train,
    labels=y_train,
    indices=indices_train,
    num_samples=num_samples_train,
    enc_seq_len=enc_seq_len,
    target_seq_len=output_sequence_length,
)
dataset_valid = TransformerDataset(
    data=X_valid,
    labels=y_valid,
    indices=indices_valid,
    num_samples=num_samples_valid,
    enc_seq_len=enc_seq_len,
    target_seq_len=output_sequence_length,
)
dataset_test = TransformerDataset(
    data=X_test,
    labels=y_test,
    indices=indices_test,
    num_samples=num_samples_test,
    enc_seq_len=enc_seq_len,
    target_seq_len=output_sequence_length,
)

# create dataloaders

dataloader_train = DataLoader(dataset_train, batch_size=batch_size)
dataloader_valid = DataLoader(dataset_valid, batch_size=batch_size)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size)

# create model

model = TimeSeriesTransformer(
    input_size=len(input_variables),
    dec_seq_len=enc_seq_len,
    batch_first=batch_first,
    num_predicted_features=1,
)

# create masks

src_mask = generate_square_subsequent_mask(dim1=output_sequence_length, dim2=enc_seq_len)

tgt_mask = generate_square_subsequent_mask(
    dim1=output_sequence_length, dim2=output_sequence_length
)

# Criterion and optimizer and early stopping

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
early_stopping = EarlyStopping()

Time series params: 
Input sequence lenght: 3 
Output sequence lenght: 1 
Step size: 1
Model hyperparameters: 
Batch size: 4 
Learning rate: 0.0001
Number of training samples: 80 
Number of valid samples: 58 
Number of test samples: 58


In [6]:
# transfer to GPU

train_on_gpu = torch.cuda.is_available()
if train_on_gpu:
    print("Training on GPU")
else:
    print("No GPU available, training on CPU")

Training on GPU


In [7]:
model.to(device)
src_mask = src_mask.to(device)
tgt_mask = tgt_mask.to(device)

In [8]:
src, trg, trg_y = next(iter(dataloader_train))
src = src.to(device)
trg = trg.to(device)
trg_y = trg_y.to(device)

In [9]:
pred = model(src=src, tgt=trg, src_mask=src_mask, tgt_mask=tgt_mask)

In [11]:
trg_y.shape

torch.Size([4, 1])